In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType
from pyspark.sql import Row
import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession \
.builder \
.appName("mongodbtest1") \
.master('local')\
.config("spark.mongodb.input.uri", "mongodb://localhost:27017/bdata.scored_reviews_bart") \
.config("spark.mongodb.output.uri", "mongodb://localhost:27017/bdata.scored_reviews_bart") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
.getOrCreate()

23/04/07 16:40:27 WARN Utils: Your hostname, Yashnas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.31.9.97 instead (on interface en0)
23/04/07 16:40:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/yashnagogineni/opt/anaconda3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/yashnagogineni/.ivy2/cache
The jars for the packages stored in: /Users/yashnagogineni/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e0680739-0ce5-46ac-84bb-b450a129ab99;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 108ms :: artifacts dl 3ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules         

23/04/07 16:40:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/07 16:40:39 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
reviews = spark.read\
.format('com.mongodb.spark.sql.DefaultSource')\
.option( "uri", "mongodb://localhost:27017/bdata.scored_reviews_bart") \
.load()

In [5]:
# select column in loaded 
reviews1 = reviews.select(['listing_id', 'reviewer_id', 'rating_score'])
# cast column datatype as float
reviews1 = reviews1.withColumn('listing_id', col('listing_id').cast(FloatType()))
reviews1 = reviews1.withColumn('reviewer_id', col('reviewer_id').cast(FloatType()))
reviews1 = reviews1.withColumn('rating_score', col('rating_score').cast(FloatType()))


In [7]:
reviews1.show(5)

+----------+-----------+------------+
|listing_id|reviewer_id|rating_score|
+----------+-----------+------------+
| 1178162.0|  4298113.0|         5.0|
| 1178162.0|  8197342.0|         5.0|
| 1178162.0|  9040491.0|         5.0|
| 1178162.0|  9101576.0|         5.0|
| 1178162.0|  6449554.0|         5.0|
+----------+-----------+------------+
only showing top 5 rows



In [8]:
# split data
train_data, test_data = reviews1.randomSplit([0.7, 0.3], seed=0)

In [9]:
# get no of records in training data
train_data.count(), test_data.count()

(45049, 19342)

In [12]:
als = ALS(
    rank=10,
    maxIter=10,
    regParam=0.1,
    userCol="listing_id",
    itemCol="reviewer_id",
    ratingCol="rating_score",
    coldStartStrategy="drop"
)

In [13]:
# Fit the model on the training data
model = als.fit(train_data)

# Generate predictions on the test data
predictions = model.transform(test_data)


23/04/07 16:43:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/07 16:43:11 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/04/07 16:43:12 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/07 16:43:12 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
# Get count of null values in each column
null_counts = reviews1.select([sum(col(c).isNull().cast("int")) for c in reviews1.columns])

# Show the result
null_counts.show()

TypeError: Column is not iterable

In [15]:
for c in reviews1.columns:
    print(reviews1.select([sum(col(c).isNull)]))

TypeError: 'method' object is not iterable